**LV-posterior-01**

Demonstration of using Bayesian model with MCMC to estimate the posterior distribution of parameters and initial conditions for a Lotka-Volterra system.

In [2]:
using DifferentialEquations, Plots
# using PyPlot
# pyplot()
# using GR
# gr()
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/airwin/.julia/packages/Plots/cc8wh/src/backends.jl:363


Plots.PlotlyBackend()

In [2]:
# Pkg.build("GR")

First define the differential equation, find a numerical solution, and plot the data.

In [3]:
function grover!(du, u, p, t)
  R, Q, X = u
  Km, Vmax, Qmin, muMax = p
  d = 0.0
  R0 = 0.0
  rho = Vmax * R / (Km + R)
  mu = muMax * (1 - Qmin/Q)
  du[1] = dRdt = d*(R0 - R) - rho*X
  du[2] = dQdt = rho - mu*Q
  du[3] = dXdt = (mu - d)*X
end

# Initial condition
u0 = [1.0, 1.0, 1.0]

# Simulation interval and intermediary points
tspan = (0.0, 10.0)
tsteps = (0.0, 1.0, 4.0, 6.0, 8.0, 10.0)

p = [0.1, 2.0, 1.0, 0.8]

# Setup the ODE problem, then solve
prob = ODEProblem(grover!, u0, tspan, p)
sol = solve(prob, Tsit5())
sol_discrete = solve(prob, Tsit5(), saveat = tsteps);

Add some measurement error to the data.

In [7]:
data1 = Array(sol_discrete) .* (1 .+ 0.05*randn(size(Array(sol_discrete))));
Plots.plot(sol)
Plots.scatter!(sol_discrete.t, data1')

<!DOCTYPE html>
 
 
 Plots.jl

Set up for using the Bayes/MCMC tools

In [9]:
using Turing, Distributions, DifferentialEquations 

# Import MCMCChain, Plots, and StatsPlots for visualizations and diagnostics.
using MCMCChains, Plots, StatsPlots

# Set a seed for reproducibility.
using Random
Random.seed!(14);
using Logging
Logging.disable_logging(Logging.Warn)

Turing.setadbackend(:forwarddiff)


:forwarddiff

Write a function to describe a model for the parameters and initial conditions.

In [17]:
@model function fitGrover(t, obs_data, problem)
    σ1 ~ InverseGamma(2, 3) # ~ is the tilde character
    σ2 ~ InverseGamma(2, 3) 
    σ3 ~ InverseGamma(2, 3) 
    R0 ~ truncated(Normal(1, 1), 0, 10)
    Q0 ~ truncated(Normal(1, 1), 0, 10)
    X0 ~ truncated(Normal(1, 1), 0, 10)
    Km ~ truncated(Normal(1,1),0,3)
    Vmax ~ truncated(Normal(1.2,0.5),0,3)
    Qmin ~ truncated(Normal(1.0,0.5),0,3)
    muMax ~ truncated(Normal(1.0,0.5),0,3)

    p = [ Km, Vmax, Qmin, muMax]

    # must define the problem with numeric values first, then update with distributions
    # prob1 = ODEProblem(droop!, [1.0, 1.0, 1.0], (0.0, 10.0), [200.0, 1.0, 1.0, 1.0])
    prob = remake(problem, u0=[R0, Q0, X0], p=p)  # modifies the original problem  # fails ****

    # prob = ODEProblem(droop!, [R0, Q0, X0], (0,10), p)
    # prob = ODEProblem(droop!, [R[1], Q[1], exp(X[1])], (0.0, 10.0), p)
    predicted = solve(prob, Rosenbrock23(), saveat=t)
    
    for j = 1:length(t)
        obs_data[1,j] ~ Normal(predicted[j][1], σ1)
        obs_data[2,j] ~ Normal(predicted[j][2], σ2)
        obs_data[3,j] ~ Normal(predicted[j][3], σ3)
    end
end


fitGrover (generic function with 3 methods)

Now use the model to define a problem for Turing.

In [18]:
problem = ODEProblem(grover!, u0, tspan, p)
model = fitGrover(sol_discrete.t, data1, problem)
chain2 = sample(model, NUTS(.65), MCMCThreads(), 250, 4, progress=false) # not enough iterations; demo only


Chains MCMC chain (250×22×4 Array{Float64,3}):

Iterations        = 1:250
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 250
parameters        = Km, Q0, Qmin, R0, Vmax, X0, muMax, σ1, σ2, σ3
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

          Km    1.0272    0.6737     0.0213    0.0413   151.8679    1.0092
          Q0    1.0918    0.3822     0.0121    0.0217   564.0120    1.0076
        Qmin    1.0256    0.1944     0.0061    0.0076   703.2759    1.0046
          R0    0.8524    0.3034     0.0096    0.0113   524.4129    1.0056
        Vmax    1.3442    0.4507     0.0143    0.0232   407.3312    0.9996
          X0    1.1383    0.349

In [29]:
reshape(Array(chain2[:R0]), 1, :)

1×1000 Array{Float64,2}:
 0.507878  1.09152  0.673157  0.822073  …  1.05256  0.713014  1.16979

Plot solution for posterior median parameters, several draws from the distribution, and the data.

In [19]:
chain_array = Array(chain2)
for k in 1:100
    u0p = chain_array[rand(1:(size(chain_array)[1])), 1:6]
    resol = solve(remake(problem, u0 = u0p[1:2], p = u0p[3:end]), Tsit5())
    Plots.plot!(resol, alpha=0.4, color = "#BBBBBB", legend = false)
end
using Statistics
u0p = median(chain_array, dims=1 )
resol = solve(remake(problem, u0 = u0p[1:2], p = u0p[3:end]), Tsit5())
Plots.plot!(sol, w=1, legend = false, lw = 2) # original solution
Plots.plot!(resol, w=1, legend = false, lw = 1) # posterior median solution
Plots.scatter!(sol_discrete.t, data1')

LoadError: [91mBoundsError: attempt to access 2-element Array{Float64,1} at index [3][39m